We're using Python 3 for this. Choose from the new notebook _py3_, not Python 2 or py2.

In [19]:
import csv

data = csv.DictReader( open('eurobaro.csv', 'r', encoding='latin1' ) )

## we need something called labels (what we try to explain) and
## something we call the features (with what we try to explain)

d = []

for row in data:
    
    f = []
    
    leftright = row['d1'] ## this is TEXT
    leftright = leftright.replace('Box', '').replace('- left', '').replace('- right', '').replace('DK', '-1').replace('Refusal', '-1')
    leftright = leftright.strip()
    leftright = int( leftright ) >= 6
    
    sex = row['d10'] == 'Woman' ## man/woman
    age = int( row['d11'] ) > 40
        
    area_rural = False
    area_town = False
    area_city = False
        
    if row['d25'] == 'Rural area or village':
        area_rural = True
            
    if row['d25'] == 'Small or middle sized town':
        area_town = True
            
    if row['d25'] == 'Large town':
        area_city = True
        
    bill_paying = False
        
    if row['d60'] == 'Most of the time': ## todo: add rest
        bill_paying = True
            
    _temp = {
        'Very satisfied' : 1,
        'Fairly satisfied' : 2,
        'Not very satisfied' : 3,
        'Not at all satisfied' : 4,
        'DK' : 2.5 ## mean!
    }
        
    live_satisfaction = _temp[ row['d70'] ] < 2.5
    
    ## now follows a hack that allows us to use this dummy representation (i.e. not Orange3 data table)
    ## we change each dummy (binary) variable into an unique number
    
    dd = []
    
    ## check each variable and add corresponding unique number value to items
    
    for i, value in enumerate( [ leftright, sex, age, area_rural, area_town, area_city, bill_paying, live_satisfaction ] ):
        
        if value:
            dd.append( i * 10 + 1 )
        else:
            dd.append( i * 10 )
        
    d.append( dd )

    ## hack ends, now if user is woman, that is indicated by number 1 only

## mapper: number values to meaninful indicators

_mapper = {}

for i, value in enumerate( [ 'leftright', 'sex', 'age', 'area_rural', 'area_town', 'area_city', 'bill_paying', 'live_satisfaction' ] ):
    
    _mapper[ i * 10 + 1 ] = value + '=True'
    _mapper[ i * 10 ] = value + '=False'

def mapper( list ):
    
    return [ _mapper[i] for i in list ]
    
data = d

In [55]:
import orangecontrib.associate.fpgrowth as assorule ## using orange3-associate, see http://orange3-associate.readthedocs.io/en/latest/#
itemsets = assorule.frequent_itemsets( data  , 2) ## minimun *support* is 2 observations

itemsets = dict( itemsets )
rules = assorule.association_rules( itemsets, .75 ) ## minimun confidence is 75%

for rule in assorule.rules_stats( rules, itemsets, len( data ) ):
    
    atecedent = ', '.join( mapper( rule[0] ) )
    consequent  = ', '.join( mapper( rule[1] ) )
    
    support = rule[2] / len( data ) ## % of cases obseerved
    confidence = rule[3]
    lift = rule[6]
    
    print( lift )
    
    if lift > 1:
        
        print( atecedent, '=>', consequent, ' ', lift, ' ', confidence)     

1.0779716527520256
leftright=True, area_town=False, sex=True, area_city=False, age=True, bill_paying=False, area_rural=False => live_satisfaction=True   1.0779716527520256   0.871313672922252
1.0779716527520256
leftright=True, sex=True, area_city=False, age=True, bill_paying=False, area_rural=False => area_town=False, live_satisfaction=True   1.0779716527520256   0.871313672922252
1.0616625401957096
leftright=True, area_town=False, sex=True, area_city=False, age=True, area_rural=False => bill_paying=False, live_satisfaction=True   1.0616625401957096   0.8118234804329725
1.0616625401957096
leftright=True, area_city=False, sex=True, age=True, area_rural=False => area_town=False, bill_paying=False, live_satisfaction=True   1.0616625401957096   0.8118234804329725
1.0
1.074826919490005
leftright=True, live_satisfaction=True, area_town=False, sex=True, area_city=False, age=True, area_rural=False => bill_paying=False   1.074826919490005   0.9663032705649157
1.074826919490005
leftright=True, l